# Micromanager Camera

Notebook for developing a microscope-gym micromanager Camera class.

Base class stores settings: 

```python
    pixel_size_um: float = Field(..., ge=0.0, description="physical pixel size of camera chip in µm")
    width_pixels: int = Field(..., ge=0, description="width of camera chip in pixels")
    height_pixels: int = Field(..., ge=0, description="height of camera chip in pixels")
    exposure_time_ms: float = Field(100.0, ge=0.0)
    gain: float = Field(0.0, ge=0.0, description="amplifier gain")
```

In [1]:
from microscope_gym.microscope_adapters.micromanager import Camera, CameraSettings

from pymmcore_plus import CMMCorePlus, Device, find_micromanager

from pathlib import Path

import stackview
import skimage

In [2]:
micromanager_path = Path(find_micromanager())
micromanager_path

PosixPath('/Applications/Micro-Manager')

In [3]:
config_file = 'MMConfig_demo.cfg'

In [4]:
microscope_handler = CMMCorePlus().instance()
microscope_handler

In [5]:
microscope_handler.loadSystemConfiguration((micromanager_path / config_file))

In [6]:
configuration = microscope_handler.getSystemState()

In [7]:
configuration.dict()

{'Camera': {'AllowMultiROI': '0',
  'AsyncPropertyDelayMS': '2000',
  'AsyncPropertyFollower': '',
  'AsyncPropertyLeader': '',
  'Binning': '1',
  'BitDepth': '16',
  'CCDTemperature': '0.0000',
  'CCDTemperature RO': '0.0000',
  'CameraID': 'V1.0',
  'CameraName': 'DemoCamera-MultiMode',
  'Description': 'Demo Camera Device Adapter',
  'DisplayImageNumber': '0',
  'DropPixels': '0',
  'Exposure': '10.0000',
  'FastImage': '0',
  'FractionOfPixelsToDropOrSaturate': '0.0020',
  'Gain': '0',
  'HubID': '',
  'MaximumExposureMs': '10000.0000',
  'Mode': 'Artificial Waves',
  'MultiROIFillValue': '0',
  'Name': 'DCam',
  'Offset': '0',
  'OnCameraCCDXSize': '512',
  'OnCameraCCDYSize': '512',
  'Photon Conversion Factor': '1.0000',
  'Photon Flux': '50.0000',
  'PixelType': '16bit',
  'ReadNoise (electrons)': '2.5000',
  'ReadoutTime': '0.0000',
  'RotateImages': '0',
  'SaturatePixels': '0',
  'ScanMode': '1',
  'SimulateCrash': '',
  'StripeWidth': '1.0000',
  'TestProperty1': '0.0000',

In [8]:
# just return the label of the CameraDevice as a string
# could be useful, but we want a device object
microscope_handler.getCameraDevice()

'Camera'

In [22]:
# device object
camera_device = Device(
    microscope_handler.getCameraDevice(),
    microscope_handler
)
camera_device

<Device 'Camera' (DemoCamera::DCam) on CMMCorePlus at 0x127a883b0: 47 properties>

In [24]:
camera_device.schema()

{'title': 'DCam',
 'description': 'Demo camera',
 'type': 'object',
 'properties': {'AllowMultiROI': {'type': 'boolean'},
  'AsyncPropertyDelayMS': {'type': 'integer'},
  'AsyncPropertyFollower': {'type': 'string', 'readOnly': True, 'default': ''},
  'AsyncPropertyLeader': {'type': 'string'},
  'Binning': {'type': 'integer', 'enum': [1, 2, 4, 8]},
  'BitDepth': {'type': 'integer', 'enum': [10, 11, 12, 14, 16, 32, 8]},
  'CCDTemperature': {'type': 'number', 'minimum': -100.0, 'maximum': 10.0},
  'CCDTemperature RO': {'type': 'number', 'readOnly': True, 'default': 0.0},
  'CameraID': {'type': 'string', 'readOnly': True, 'default': 'V1.0'},
  'CameraName': {'type': 'string',
   'readOnly': True,
   'default': 'DemoCamera-MultiMode'},
  'Description': {'type': 'string',
   'readOnly': True,
   'default': 'Demo Camera Device Adapter'},
  'DisplayImageNumber': {'type': 'boolean'},
  'DropPixels': {'type': 'boolean'},
  'Exposure': {'type': 'number', 'minimum': 0.0, 'maximum': 10000.0},
  'Fa

In [10]:
camera_device.getPropertyObject('OnCameraCCDXSize').value

512

In [11]:
camera_settings = CameraSettings(
            pixel_size_um=microscope_handler.getPixelSizeUm(),
            width_pixels=camera_device.getPropertyObject(
            'OnCameraCCDXSize').value,
            height_pixels=camera_device.getPropertyObject(
            'OnCameraCCDYSize').value,
            exposure_time_ms=camera_device.getPropertyObject(
            'Exposure' ).value,
            gain=camera_device.getPropertyObject(
            'Gain').value
        )
camera_settings

CameraSettings(pixel_size_um=1.0, width_pixels=512, height_pixels=512, exposure_time_ms=10.0, gain=0.0)

In [12]:
microscope_handler.getPixelSizeUm()

1.0

In [13]:
camera = Camera(microscope_handler)
camera

/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/microscope-gym/microscope_gym/microscope_adapters/micromanager.py:103: UserWarning: Pixel size is set to 1.0; it may not be set or calibrated.
  warn(f'Pixel size is set to {self._settings.pixel_size_um}; it may not be set or calibrated.')


In [14]:
image = camera.capture_image()
image

array([[3276, 3339, 3402, ..., 3087, 3150, 3213],
       [3284, 3347, 3410, ..., 3095, 3158, 3221],
       [3292, 3355, 3418, ..., 3103, 3166, 3229],
       ...,
       [5846, 5846, 5844, ..., 5837, 5842, 5845],
       [5846, 5846, 5844, ..., 5838, 5842, 5845],
       [5846, 5846, 5843, ..., 5839, 5843, 5845]], dtype=uint16)

In [15]:
type(image)

numpy.ndarray

In [16]:
# some problems viewingin pycharm, fix later
# stackview.insight(image)
# skimage.io.imshow(image)

In [17]:
# try to set some camera properties
settings = camera.settings.model_copy() # model_copy is pydantic
settings.width_pixels = 888              # try a new value
camera.configure_camera(settings)       #set the new value

In [19]:
camera.settings

CameraSettings(pixel_size_um=1.0, width_pixels=888, height_pixels=512, exposure_time_ms=10.0, gain=0.0)